# TensorFlow queues and input pipelines

## Initialization

In [24]:
import numpy as np
import tensorflow as tf
import threading
import time

import os
from cStringIO import StringIO
import numpy as np
from functools import partial

from IPython.display import clear_output, Image, display, HTML

import tensorflow as tf

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
def create_session():
    sess = tf.InteractiveSession(config=tf.ConfigProto(operation_timeout_in_ms=3000))
    return sess


In [25]:
tf.reset_default_graph()
a = tf.constant(1)
b = tf.constant(2)
c = a+b
d = c*a
e = d-c

In [26]:
show_graph(tf.get_default_graph().as_graph_def())

# Simple queue

In [27]:
tf.reset_default_graph()
q = tf.FIFOQueue(capacity=20, dtypes=[tf.int32])
enqueue_placeholder = tf.placeholder(tf.int32)
enqueue_op = q.enqueue(enqueue_placeholder)
sess = create_session()
for i in range(10):
    sess.run(enqueue_op, feed_dict={enqueue_placeholder:i})
    print "Queue size is now: "+str(sess.run(q.size()))
sess.run(q.close())

Queue size is now: 1
Queue size is now: 2
Queue size is now: 3
Queue size is now: 4
Queue size is now: 5
Queue size is now: 6
Queue size is now: 7
Queue size is now: 8
Queue size is now: 9
Queue size is now: 10


In [5]:
try:
    for i in range(20):
        print sess.run(q.dequeue())
except tf.errors.OutOfRangeError:
    print "Done"

0
1
2
3
4
5
6
7
8
9
Done


# Simple queue with multiple enqueues in parallel

In [6]:
tf.reset_default_graph()
random_number = tf.random_uniform(shape=())
q = tf.FIFOQueue(capacity=20, dtypes=[tf.float32])
enqueue_op = q.enqueue(random_number)

sess = create_session()
print sess.run(q.size())
def run():
  for i in range(5):
    sess.run(enqueue_op)

threads = [threading.Thread(target=run, args=()) for i in range(2)]
[t.start() for t in threads]
print sess.run(q.size())
time.sleep(0.5)
print sess.run(q.size())


0
0


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf8bb7a10>> ignored


10


# Setting up range_input_producer

In [7]:
inn = tf.train.range_input_producer(limit=10, num_epochs=1, shuffle=False)

In [8]:
show_graph(tf.get_default_graph().as_graph_def())

## Lookup name of size operation in GraphDef

If you don't want to create new Node with "op.size", you can find name of exiting node by looking for with op="QueueSize" in the graph definition, and fetching it directly. In this case, proper run call will look like this -- sess.run("input_producer/fraction_of_32_full/fraction_of_32_full_Size:0")

In [9]:
tf.get_default_graph().as_graph_def()

node {
  name: "random_uniform/shape"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
          }
        }
      }
    }
  }
}
node {
  name: "random_uniform/min"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 0.0
      }
    }
  }
}
node {
  name: "random_uniform/max"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "random_uniform/RandomUniform"
  op: "RandomUniform"
  input: "random_uniform/shape"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
  attr {
   

In [10]:
tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS)

In [11]:
threads = tf.train.start_queue_runners()

In [12]:
sess.graph.version

32

In [13]:
sess.run(inn.size())

ERROR:tensorflow:Exception in QueueRunner: Attempting to use uninitialized value input_producer/fraction_of_32_full/limit_epochs/epochs
	 [[Node: input_producer/fraction_of_32_full/limit_epochs/CountUpTo = CountUpTo[T=DT_INT64, limit=1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer/fraction_of_32_full/limit_epochs/epochs)]]
Caused by op u'input_producer/fraction_of_32_full/limit_epochs/CountUpTo', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().s

0

In [14]:
sess.graph.version

33

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/training/queue_runner.py", line 185, in _run
    sess.run(enqueue_op)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 340, in run
    run_metadata_ptr)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 564, in _run
    feed_dict_string, options, run_metadata)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 637, in _do_run
    target_list, options, run_metadata)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 659, in _do_call
    e.code)
FailedPreconditionError: Attempting to use uninitialized value inpu

In [15]:
"input_producer/fraction_of_32_full/fraction_of_32_full_Size:0"

'input_producer/fraction_of_32_full/fraction_of_32_full_Size:0'

# Alternatives to "wait forever"

## Set session timeout

In [16]:
tf.reset_default_graph()
queue = tf.FIFOQueue(capacity=5, dtypes=[tf.int32])
config = tf.ConfigProto()
config.operation_timeout_in_ms=2000
sess = tf.InteractiveSession("", config=config)
try:
    sess.run(queue.dequeue())
except tf.errors.DeadlineExceededError:
    print "DeadlineExceededError"

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf8b95390>> ignored


DeadlineExceededError


## Close the queue before reading

In [17]:
tf.reset_default_graph()
queue = tf.FIFOQueue(capacity=5, dtypes=[tf.int32])
config = tf.ConfigProto()
config.operation_timeout_in_ms=2000
sess = tf.InteractiveSession("", config=config)
sess.run(queue.close())
try: 
    sess.run(queue.dequeue())
except tf.errors.OutOfRangeError as e:
    print "OutOfRangeError"

OutOfRangeError


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf4259b90>> ignored


## Close the queue before writing

In [18]:
tf.reset_default_graph()
queue = tf.FIFOQueue(capacity=5, dtypes=[tf.int32])
config = tf.ConfigProto()
config.operation_timeout_in_ms=2000
sess = tf.InteractiveSession("", config=config)
sess.run(queue.close())
try:
    sess.run(queue.enqueue(5))
except tf.errors.AbortedError:
    print "AbortedError"

AbortedError


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf4259ad0>> ignored


# Simple batching example

In [19]:
tf.reset_default_graph()
ranges = tf.train.range_input_producer(limit=10, num_epochs=1, shuffle=False)
number = ranges.dequeue()
numeric_batch = tf.train.batch_join([[number]]*3, batch_size=2)
batch_number = numeric_batch

sess = tf.InteractiveSession(config=tf.ConfigProto(operation_timeout_in_ms=6000))
sess.run(tf.initialize_all_variables())
tf.train.start_queue_runners()

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf4259c50>> ignored


[<Thread(Thread-8, started daemon 140384554444544)>,
 <Thread(Thread-9, started daemon 140384546051840)>,
 <Thread(Thread-10, started daemon 140384537659136)>,
 <Thread(Thread-11, started daemon 140384529266432)>]

In [20]:
tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS)

### Python/TensorFlow gotcha

In [21]:
tf.reset_default_graph()
ranges = tf.train.range_input_producer(limit=10, num_epochs=1, shuffle=False)
number = ranges.dequeue()
sess = tf.InteractiveSession(config=tf.ConfigProto(operation_timeout_in_ms=6000))
sess.run(tf.initialize_all_variables())
tf.train.start_queue_runners()

print sess.run([number]*3)

[0, 0, 0]


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf4259ad0>> ignored


### Solution 1

In [22]:
tf.reset_default_graph()
ranges = tf.train.range_input_producer(limit=10, num_epochs=1, shuffle=False)
number = ranges.dequeue()
sess = tf.InteractiveSession(config=tf.ConfigProto(operation_timeout_in_ms=6000))
sess.run(tf.initialize_all_variables())
tf.train.start_queue_runners()

print sess.run([ranges.dequeue()]*3)

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf42529d0>> ignored


[0, 0, 0]


### Solution 2

In [23]:
tf.reset_default_graph()
ranges = tf.train.range_input_producer(limit=10, num_epochs=1, shuffle=False)
number = ranges.dequeue()
sess = tf.InteractiveSession(config=tf.ConfigProto(operation_timeout_in_ms=6000))
sess.run(tf.initialize_all_variables())
tf.train.start_queue_runners()

print sess.run([ranges.dequeue(), ranges.dequeue(), ranges.dequeue()])

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fadf4205790>> ignored


NotFoundError: FetchOutputs node fraction_of_32_full_Dequeue_3:0: not found